In [3]:
#imports


from __future__ import print_function

import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [8]:
# generate data
np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')
torch.save(data, open('traindata.pt', 'wb'))

In [ ]:
# Import the data
# Data shape 5 dim vector []

In [23]:
# Sequence model

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(5, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 5)

    def forward(self, input, future = 0):
        outputs = []
        h_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [21]:
# # set random seed to 0
# np.random.seed(0)
# torch.manual_seed(0)

# load data and make training set
data = pd.read_csv('cleaned/states_2017-08-28-00.csv')
# print(data['NMAC_id'][5])

# data_nparr = data.as_matrix()
# print(data_nparr[0])


# # drop unneeded columns to form matrix to be trained on
# data_matr = data.drop(['NMAC_id', 'alert_1', 'alert_2', 'onground_1', 'onground_2'], axis=1).as_matrix().astype('float')
# # print(data_m)

# restructure data
all_planes = []

batch_size = 5
limit_timesteps = 20
count = 0
unique_icao = data.icao24.unique()
for name in unique_icao:
    # only get batch size for now
    if count > batch_size:
        break
    
    # get rows where icao == name (boolean vector)
    selector = data['icao24'] == name
    one_plane_df = data[selector]
    
#     for index, row in one_plane_df.iterrows():
        
    one_plane_df = one_plane_df.drop(['icao24','time'], axis=1)
    one_plane_arr = one_plane_df.as_matrix()
    one_plane_arr = one_plane_arr[1:limit_timesteps+1]
    
    
#     # Make sure columns have data, otherwise skip plane
#     if (((one_plane_df['x_displacement'] == 0) & (one_plane_df['y_displacement'] == 0)).any()):
#         continue
    
    all_planes.append(one_plane_arr)
    
    count += 1
    
batch = np.asarray(all_planes) # convert to np matrix
    
input = Variable(torch.from_numpy(batch[:,:,:-1]), requires_grad=False)
target = Variable(torch.from_numpy(batch[:,:,1:]), requires_grad=False)

# data = torch.load('traindata.pt')
# input = Variable(torch.from_numpy(data[3:, :-1]), requires_grad=False)
# target = Variable(torch.from_numpy(data[3:, 1:]), requires_grad=False)
# test_input = Variable(torch.from_numpy(data[:3, :-1]), requires_grad=False)
# test_target = Variable(torch.from_numpy(data[:3, 1:]), requires_grad=False)


print(batch)

[[[  1.09499400e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09499400e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09575600e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09575600e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09651800e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09728000e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09728000e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09728000e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09804200e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09804200e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00000000e+00]
  [  1.09804200e+04   3.13695768e+02   2.07764692e+02   0.00000000e+00
     0.00

In [24]:


# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()

# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)

# begin to train
for i in range(15):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.data.numpy()[0])
        loss.backward()
        return loss
    optimizer.step(closure)
    
    # begin to predict
    future = 6
    pred = seq(test_input, future = future)
    loss = criterion(pred[:, :-future], test_target)
    print('test loss:', loss.data.numpy()[0])
    y = pred.data.numpy()
    
#     # draw the result
#     plt.figure(figsize=(30,10))
#     plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
#     plt.xlabel('x', fontsize=20)
#     plt.ylabel('y', fontsize=20)
#     plt.xticks(fontsize=20)
#     plt.yticks(fontsize=20)
#     def draw(yi, color):
#         plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
#         plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
#     draw(y[0], 'r')
#     draw(y[1], 'g')
#     draw(y[2], 'b')
#     plt.savefig('predict%d.pdf'%i)
#     plt.close()


STEP:  0


RuntimeError: size mismatch, m1: [6 x 4], m2: [5 x 204] at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/generic/THTensorMath.c:1293